# **GCD_v1 algorithm in with Vlad's lib**
[The detailed algorithm](https://web.eecs.umich.edu/~weimerw/2015-4610/scottcd/10a_thfnd.pdf)
[Helped Wiki](https://en.wikipedia.org/wiki/Lambda_calculus)

1) gcd ≡ λa.λb.(if (equal a b) a (if (greater than a b) (gcd (minus a b) b) (gcd (Minus b a) a)))
2) gcd ≡ (λg .λa.λb.(if (equal a b) a (if (greater than a b) (g (minus a b) b) (g (minus b a) a)))) gcd
3) gcd ≡ f gcd
4) λg .λa.λb.(if (equal a b) a (if (greater than a b) (g (minus a b) b) (g (minus b a) a)))
5) gcd ≡ (λh.(λx.h(x x)) (λx.h(x x)))
        (λg .λa.λb.(if (equal a b) a
        (if (greater than a b) (g (minus a b) b) (g (minus b a) a))))
6) Y ≡ (λh.(λx.h(x x)) (λx.h(x x)))
7) gcd ≡ Y
        (λg .λa.λb.(if (equal a b) a
        (if (greater than a b) (g (minus a b) b) (g (minus b a) a))))


In [1]:
import sys

sys.path.append("../")
from calculus.term import *
from calculus.strategy import *
from calculus.advanced_terms import *

gcd ≡ (Y
        (λg.λa.λb.(ite (equal a b) a
        (ite (greater_than a b) (g (minus a b) b) (g (minus b a) a))))
        )

In [2]:
# PRED := λn.λf.λx.n (in_term_1) (in_term_0) (i_term())
# in_term_0 = λu.x
# in_term_1 = λg.λh.h (g f)
def w_pred_term():
    n, f, x, h, g, u = Var(), Var(), Var(), Var(), Var(), Var()
    x_, n_ = Atom(x), Atom(n)
    h_, g_, f_ = Atom(h), Atom(g), Atom(f)
    in_term_0 = Lambda(u, x_)
    in_term_1 = Lambda(g, Lambda(h, App(h_, App(g_, f_))))
    return Lambda(n, Lambda(f, Lambda(x, multi_app_term(n_, in_term_1, in_term_0, i_term()))))

In [3]:
def minus_term():
    m, n = Var(), Var()
    m_, n_ = Atom(m), Atom(n)
    return Lambda(m, Lambda(n, multi_app_term(n_, w_pred_term(), m_)))

In [6]:
strategy = RightmostInnermostStrategy()

term = App(w_pred_term(), n_term(0))
print(term.normalize(strategy)[0].funky_str())

term = App(w_pred_term(), n_term(1))
print(term.normalize(strategy)[0].funky_str())

term = App(w_pred_term(), n_term(4))
print(term.normalize(strategy)[0].funky_str())

term = App(w_pred_term(), n_term(10))
print(term.normalize(strategy)[0].funky_str())

λq.λw.w
λq.λw.w
λq.λw.(q (q (q w)))
λq.λw.(q (q (q (q (q (q (q (q (q w)))))))))


In [9]:
strategy = LeftmostOutermostStrategy()

term = multi_app_term(minus_term(), n_term(0), n_term(0))
print(term.normalize(strategy)[0].funky_str())

term = multi_app_term(minus_term(), n_term(3), n_term(0))
print(term.normalize(strategy)[0].funky_str())

term = multi_app_term(minus_term(), n_term(0), n_term(3))
print(term.normalize(strategy)[0].funky_str())

term = multi_app_term(minus_term(), n_term(4), n_term(2))
print(term.normalize(strategy)[0].funky_str())

λq.λw.w
λq.λw.(q (q (q w)))
λw.λe.λq.e
λq.λw.w


gcd ≡ (Y
        (λg.λa.λb.(ite (equal a b) a
        (ite (greater_than a b) (g (minus a b) b) (g (minus b a) a) )))
        )


In [14]:
def gcd_v2():
    g, a, b = Var(), Var(), Var()
    g_, a_, b_ = Atom(g), Atom(a), Atom(b)
    equal_ab = multi_app_term(eq_term(), a_, b_)

    minus_ab = multi_app_term(g_, multi_app_term(minus_term(), a_, b_), b_)
    minus_ba = multi_app_term(g_, multi_app_term(minus_term(), b_, a_), a_)
    le_ab  = multi_app_term(le_term(), b_, a_)
    inner_ite_term = multi_app_term(ite_term(), le_ab, minus_ab, minus_ba)

    ite_main_term = multi_app_term(ite_term(), equal_ab, a_, inner_ite_term)
    gcd_lambda = Lambda(g, Lambda(a, Lambda(b, ite_main_term)))
    return App(y_term(), gcd_lambda)

In [17]:
def test_gcd(var_0 = 0, var_1 = 0, strategy = LeftmostOutermostStrategy()):
    term_ = multi_app_term(gcd_v2(), n_term(var_0), n_term(var_1))
    res_term, steps = term_.normalize_no_lim(strategy)

    print(term_.funky_str(), "\n\n")
    print(steps)
    print(res_term.funky_str())

In [18]:
test_gcd(0, 0, LeftmostOutermostStrategy())

KeyboardInterrupt: 